In [1]:
from data_loader import OpenImagesDataset, collate_fn
from model_utils import plot_tensor, plot_image
from model_transformations import Transformations
from torch.utils.data import DataLoader
from params import DEVICE
from model import DogDetectorModel
import torch
from model_trainer import trainer
from model_evaluator import evaluator

In [2]:
# Initializing the model
model = DogDetectorModel()

# Moving to training device
model = model.to(DEVICE)

In [3]:
BATCH_SIZE = 2

In [4]:
# Reading in the training data
trainingData = OpenImagesDataset(rootDirectory='open-images-v6',
                                 transform=Transformations, 
                                 dataType='train')    

# Defining the training data
trainDataLoader = DataLoader(dataset=trainingData, 
                             batch_size=BATCH_SIZE,
                             num_workers=4,
                             shuffle=False, collate_fn=collate_fn)

# Reading in the training data
validationData = OpenImagesDataset(rootDirectory='open-images-v6',
                                 transform=Transformations, 
                                 dataType='validation')    

# Defining the training data
validationDataLoader = DataLoader(dataset=validationData, 
                             batch_size=1,
                             num_workers=2,
                             shuffle=False, collate_fn=collate_fn)

In [5]:
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# Defining the learning rate that makes a step every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [6]:
NUM_EPOCHS = 10

In [ ]:
for epoch in range(NUM_EPOCHS):
    # ***************** TRAINING ******************    

    trainer(model, optimizer, trainDataLoader, epoch)
    
    # Updating the learning rate scheduler
    lr_scheduler.step()
    
    # ***************** EVALUATION ******************    

    x = evaluator(model, validationDataLoader)
    
   


  0%|                                                                                                                                                                                                                                                          | 0/500 [00:00<?, ?it/s]/home/aidan/Programs/miniconda3/envs/dogapp/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811803361/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
EPOCH: 0 | Classifier Loss 0.03860253468155861 | Bbox Loss: 0.05920293554663658 | Objectness Loss 0.005157435312867165 | Loss 0.11637904495000839: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [02:38<00:00,  3.16it/s]


creating index...
index created!


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 14.07it/s]

Accumulating evaluation results...
DONE (t=0.00s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.900
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.900
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.900
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.900
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.900
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area


EPOCH: 1 | Classifier Loss 0.024352915585041046 | Bbox Loss: 0.040665872395038605 | Objectness Loss 0.0009331000037491322 | Loss 0.06892501562833786:  38%|██████████████████████████████████                                                        | 189/500 [00:57<01:29,  3.47it/s]

In [ ]:
PATH = "model.pt"
torch.save({
'epoch': EPOCH,
'model_state_dict': mdoel.state_dict(),
'optimizer_state_dict': optimizer.state_dict(),
'loss': -1,
}, PATH)

In [ ]:
plot_tensor(images.cpu(),outputs[0]['boxes'][0].reshape((1,-1)).cpu().detach().numpy())